# Part IV: Modelling

## Basic settings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import pickle
import sys
import warnings
from datetime import datetime, timedelta

import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit

warnings.filterwarnings("ignore")

# Set plotting style
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("deep")

# Set random seed for reproducibility
np.random.seed(2025)

/home/research/miniconda3/envs/sales_forecasting_ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
src_path = os.path.abspath(os.path.join("../..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [4]:
from utils.plots import plot_forecast_single
from utils.utils import flatten_prophet_predictions, weighted_absolute_percentage_error

In [5]:
import os
import pandas as pd
import numpy as np

DATA_DIR = "../../data"

# 1. LOAD DATA ĐÃ PREPROCESS VÀ FEATURE ENGINEERING
df_sales = pd.read_csv(
    os.path.join(DATA_DIR, "data_processed/sales_data_preprocessed.csv"),
    parse_dates=["date"]
)
df_weather = pd.read_csv(
    os.path.join(DATA_DIR, "data_processed/weather_preprocessed.csv"),
    parse_dates=["date"]
)
df_weather_key_store_merged = pd.read_csv(
    os.path.join(DATA_DIR, "data_processed/weather_key_store_merged.csv"),
    parse_dates=["date"]
)

# Đây là file đã có is_kaggle_test và toàn bộ features
df_features = pd.read_feather(os.path.join(DATA_DIR,'data_processed/feature_engineered_data_89_features.feather'))

print("Full feature data:", df_features.shape)
print("Kaggle test rows:", df_features['is_kaggle_test'].sum())
print("Train rows:", (df_features['is_kaggle_test'] == 0).sum())




Full feature data: (686187, 89)
Kaggle test rows: 526917
Train rows: 159270


## Load data

In [6]:
df_features['is_valid'] = 0
mask_train = df_features['is_kaggle_test'] == 0
cutoff_date = pd.Timestamp("2014-08-01")
df_features.loc[mask_train & (df_features['date'] >= cutoff_date), 'is_valid'] = 1

# 2. Tách train/valid và kaggle test
df_train = df_features[(df_features['is_kaggle_test'] == 0) & (df_features['is_valid'] == 0)].copy()
df_valid = df_features[(df_features['is_kaggle_test'] == 0) & (df_features['is_valid'] == 1)].copy()
df_kaggle_test = df_features[df_features['is_kaggle_test'] == 1].copy()

print("Final splits:")
print("  Train:", df_train.shape)
print("  Valid:", df_valid.shape)
print("  Kaggle test:", df_kaggle_test.shape)

Final splits:
  Train: (153496, 90)
  Valid: (5774, 90)
  Kaggle test: (526917, 90)


In [7]:
df_kaggle_test

,date,store_nbr,item_nbr,units,logunits,is_kaggle_test,station_nbr,tmax,depart,cool,...,logunits_ewma_14d_a05,logunits_ewma_28d_a05,logunits_ewma_7d_a075,logunits_ewma_14d_a075,logunits_ewma_28d_a075,store_sum_7d,store_mean_7d,item_sum_7d,item_mean_7d,is_valid
159270,2013-04-01,2,1,NaN,NaN,True,14,71.0,1.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,8.670772,1.238682,7.203406,1.029058,0
159271,2013-04-01,3,1,NaN,NaN,True,7,68.0,6.200000,0.0,...,NaN,NaN,NaN,NaN,NaN,8.825560,1.260794,6.510258,1.085043,0
159272,2013-04-01,6,1,NaN,NaN,True,14,71.0,1.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,12.102488,1.728927,5.817111,1.163422,0
159273,2013-04-01,7,1,NaN,NaN,True,6,86.0,6.000000,5.0,...,NaN,NaN,NaN,NaN,NaN,8.648221,1.235460,5.123964,1.280991,0
159274,2013-04-01,8,1,NaN,NaN,True,4,87.0,8.000000,9.0,...,NaN,NaN,NaN,NaN,NaN,9.572480,1.367497,3.178054,1.059351,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686182,2014-10-26,1,111,NaN,NaN,True,1,58.0,5.666667,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,0.693147,0
686183,2014-10-26,14,111,NaN,NaN,True,16,58.0,7.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,0.693147,0
686184,2014-10-26,16,111,NaN,NaN,True,2,53.0,2.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,0.693147,0
686185,2014-10-26,19,111,NaN,NaN,True,15,57.0,4.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,0.693147,0


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153496 entries, 0 to 159249
Data columns (total 90 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    153496 non-null  datetime64[ns]
 1   store_nbr               153496 non-null  int64         
 2   item_nbr                153496 non-null  int64         
 3   units                   153496 non-null  float64       
 4   logunits                153496 non-null  float64       
 5   is_kaggle_test          153496 non-null  bool          
 6   station_nbr             153496 non-null  int64         
 7   tmax                    153496 non-null  float64       
 8   depart                  153496 non-null  float64       
 9   cool                    153496 non-null  float64       
 10  sunrise                 153496 non-null  float64       
 11  sunset                  153496 non-null  float64       
 12  snowfall                153496 non-

## Base line model

In [9]:
prophet_train_data = (
    df_train.groupby(['store_nbr', 'item_nbr', 'date'], as_index=False)
    .agg({
        'logunits': 'sum',
        'is_weekend': 'first',
        'is_holiday': 'first',
        'cool': 'mean',
        'preciptotal': 'mean'
    })
    .rename(columns={
        'date': 'ds',
        'logunits': 'y',
        'cool': 'temp',
        'preciptotal': 'rain'
    })
)

prophet_valid_data = (
    df_valid.groupby(['store_nbr', 'item_nbr', 'date'], as_index=False)
    .agg({
        'logunits': 'sum',
        'is_weekend': 'first',
        'is_holiday': 'first',
        'cool': 'mean',
        'preciptotal': 'mean'
    })
    .rename(columns={
        'date': 'ds',
        'logunits': 'y',
        'cool': 'temp',
        'preciptotal': 'rain'
    })
)

print("Prophet train shape:", prophet_train_data.shape)
print("Prophet valid shape:", prophet_valid_data.shape)

Prophet train shape: (153496, 8)
Prophet valid shape: (5774, 8)


In [10]:
prophet_train_data

,store_nbr,item_nbr,ds,y,is_weekend,is_holiday,temp,rain
0,1,9,2012-01-29,5.225747,1,0,0.0,0.000000
1,1,9,2012-01-30,4.304065,0,0,0.0,0.118415
2,1,9,2012-01-31,3.688879,0,0,0.0,0.118415
3,1,9,2012-02-01,4.110874,0,0,0.0,0.010000
4,1,9,2012-02-02,4.343805,0,0,0.0,0.000000
...,...,...,...,...,...,...,...,...
153491,45,50,2014-06-18,0.000000,0,0,12.0,0.125842
153492,45,50,2014-06-19,0.000000,0,1,5.0,0.000000
153493,45,50,2014-06-20,0.000000,0,0,0.0,0.000000
153494,45,50,2014-06-21,0.000000,1,0,0.0,0.000000


In [11]:
def build_prophet_model(prophet_train_data, prophet_valid_data):
    print("Building Prophet models...")
    
    # Lấy pairs từ TRAIN data
    pairs = prophet_train_data[['store_nbr', 'item_nbr']].drop_duplicates()
    
    prophet_predictions = {}
    prophet_models = {}
    prophet_metrics = pd.DataFrame(columns=['store_nbr', 'item_nbr', 'mae', 'rmse', 'wape'])
    all_actual, all_predicted = [], []
    
    skipped = 0
    for _, row in pairs.iterrows():
        # Ép kiểu int rõ ràng để đồng bộ với lúc dự đoán
        s, i = int(row['store_nbr']), int(row['item_nbr'])
        
        # TRAIN từ prophet_train_data
        combo_train = prophet_train_data[
            (prophet_train_data['store_nbr'] == s) & 
            (prophet_train_data['item_nbr'] == i)
        ].copy()
        
        # TEST từ prophet_valid_data  
        combo_test = prophet_valid_data[
            (prophet_valid_data['store_nbr'] == s) & 
            (prophet_valid_data['item_nbr'] == i)
        ].copy()
        
        # CHỈ SKIP NẾU KHÔNG CÓ DỮ LIỆU TRAIN
        if combo_train.empty:
            skipped += 1
            continue
            
        # Train Prophet
        m = Prophet(
            daily_seasonality=False,
            weekly_seasonality=True,
            yearly_seasonality=True,
            seasonality_mode='multiplicative'
        )
        
        for reg in ['is_weekend', 'is_holiday', 'temp', 'rain']:
            m.add_regressor(reg)
            
        m.fit(combo_train[['ds', 'y', 'is_weekend', 'is_holiday', 'temp', 'rain']])
        
        # Lưu Model (Lưu key là int)
        key = (s, i)
        prophet_models[key] = m
        
        # Chỉ tính metrics nếu có dữ liệu validation (combo_test không rỗng)
        if not combo_test.empty:
            future = combo_test[['ds', 'is_weekend', 'is_holiday', 'temp', 'rain']]
            fcst = m.predict(future)
            fcst = fcst[['ds', 'yhat']].merge(combo_test[['ds', 'y']], on='ds')
            
            prophet_predictions[key] = fcst
            
            # Metrics
            mae = mean_absolute_error(fcst['y'], fcst['yhat'])
            rmse = np.sqrt(mean_squared_error(fcst['y'], fcst['yhat']))
            wape = weighted_absolute_percentage_error(fcst['y'], fcst['yhat'])
            
            prophet_metrics.loc[len(prophet_metrics)] = [s, i, mae, rmse, wape]
            all_actual.extend(fcst['y'])
            all_predicted.extend(fcst['yhat'])
    
    print(f"Skipped: {skipped}/{len(pairs)} pairs (due to missing training data)")
    print(f"Models built: {len(prophet_models)}")
    
    # Tính metrics tổng (chỉ trên những model có validation)
    if len(prophet_metrics) > 0:
        avg_mae = prophet_metrics['mae'].mean()
        avg_rmse = prophet_metrics['rmse'].mean()
        avg_wape = weighted_absolute_percentage_error(np.array(all_actual), np.array(all_predicted))
        
        print(f"\nOverall Validation Metrics (on {len(prophet_metrics)} validated models):")
        print(f"MAE: {avg_mae:.4f} | RMSE: {avg_rmse:.4f} | WAPE: {avg_wape:.4f}")
        return prophet_models, prophet_predictions, (avg_mae, avg_rmse, avg_wape)
    else:
        print("\nWarning: No validation data available for any model.")
        return prophet_models, prophet_predictions, (0, 0, 0)

In [12]:
prophet_models, prophet_predictions, metrics = build_prophet_model(
    prophet_train_data, 
    prophet_valid_data
)

Building Prophet models...


18:42:21 - cmdstanpy - INFO - Chain [1] start processing
18:42:21 - cmdstanpy - INFO - Chain [1] done processing
18:42:21 - cmdstanpy - INFO - Chain [1] start processing
18:42:21 - cmdstanpy - INFO - Chain [1] done processing
18:42:21 - cmdstanpy - INFO - Chain [1] start processing
18:42:21 - cmdstanpy - INFO - Chain [1] done processing
18:42:21 - cmdstanpy - INFO - Chain [1] start processing
18:42:21 - cmdstanpy - INFO - Chain [1] done processing
18:42:21 - cmdstanpy - INFO - Chain [1] start processing


18:42:21 - cmdstanpy - INFO - Chain [1] done processing
18:42:21 - cmdstanpy - INFO - Chain [1] start processing
18:42:21 - cmdstanpy - INFO - Chain [1] done processing
18:42:21 - cmdstanpy - INFO - Chain [1] start processing
18:42:21 - cmdstanpy - INFO - Chain [1] done processing
18:42:21 - cmdstanpy - INFO - Chain [1] start processing
18:42:22 - cmdstanpy - INFO - Chain [1] done processing
18:42:22 - cmdstanpy - INFO - Chain [1] start processing
18:42:22 - cmdstanpy - INFO - Chain [1] done processing
18:42:22 - cmdstanpy - INFO - Chain [1] start processing
18:42:22 - cmdstanpy - INFO - Chain [1] done processing
18:42:22 - cmdstanpy - INFO - Chain [1] start processing
18:42:22 - cmdstanpy - INFO - Chain [1] done processing
18:42:22 - cmdstanpy - INFO - Chain [1] start processing
18:42:22 - cmdstanpy - INFO - Chain [1] done processing
18:42:22 - cmdstanpy - INFO - Chain [1] start processing
18:42:22 - cmdstanpy - INFO - Chain [1] done processing
18:42:22 - cmdstanpy - INFO - Chain [1] 

Skipped: 0/255 pairs (due to missing training data)
Models built: 255

Overall Validation Metrics (on 205 validated models):
MAE: 0.3909 | RMSE: 0.4720 | WAPE: 24.5412


In [13]:
# Accuracy of Prophet Model
print(
    f"Prophet Model Results:\nMAE: {metrics[0]:.2f} | RMSE: {metrics[1]:.2f} | WAPE: {metrics[2]:.2f}%"
)

Prophet Model Results:
MAE: 0.39 | RMSE: 0.47 | WAPE: 24.54%


In [ ]:
# 1. Lọc đúng dữ liệu cho tập Test (từ 01/04/2013 trở đi)
min_test_date = "2013-04-01"

df_kaggle_test_prophet = df_kaggle_test[
    (df_kaggle_test['is_kaggle_test'] == True) & 
    (df_kaggle_test['date'] >= min_test_date)
].copy()

df_kaggle_test_prophet = df_kaggle_test_prophet[[
    'store_nbr', 'item_nbr', 'date',
    'is_weekend', 'is_holiday', 'cool', 'preciptotal'
]].rename(columns={
    'date': 'ds',
    'cool': 'temp',
    'preciptotal': 'rain'
})

# 2. Dự đoán vòng lặp từng Store-Item
kaggle_preds = []
pairs_test = df_kaggle_test_prophet[['store_nbr','item_nbr']].drop_duplicates()

for _, row in pairs_test.iterrows():
    s, i = int(row['store_nbr']), int(row['item_nbr'])
    key = (s, i)

    df_si = df_kaggle_test_prophet[
        (df_kaggle_test_prophet['store_nbr'] == s) &
        (df_kaggle_test_prophet['item_nbr'] == i)
    ].copy()

    if key not in prophet_models:
        df_si['yhat'] = 0.0
    else:
        m = prophet_models[key]
        future = df_si[['ds','is_weekend','is_holiday','temp','rain']]
        fcst = m.predict(future)
        df_si = df_si.merge(fcst[['ds','yhat']], on='ds', how='left')

    kaggle_preds.append(df_si)

# 3. Gộp và xử lý kết quả dự đoán
df_kaggle_pred = pd.concat(kaggle_preds, ignore_index=True)
df_kaggle_pred['units'] = np.expm1(df_kaggle_pred['yhat']).clip(lower=0)
df_kaggle_pred['date_str'] = df_kaggle_pred['ds'].dt.strftime('%Y-%m-%d')

# 4. Sắp xếp theo yêu cầu Kaggle và tạo ID
df_kaggle_pred = df_kaggle_pred.sort_values(['date_str', 'store_nbr', 'item_nbr'])
df_kaggle_pred['id'] = (
    df_kaggle_pred['store_nbr'].astype(str) + '_' +
    df_kaggle_pred['item_nbr'].astype(str) + '_' +
    df_kaggle_pred['date_str']
)

submission = df_kaggle_pred[['id', 'units']].reset_index(drop=True)
submission.to_csv('submission_prophet.csv', index=False)

print(f"✓ Saved submission_prophet.csv ({len(submission)} rows)")
print("Sample IDs:", submission['id'].head().tolist())


✓ Saved submission_prophet.csv (526917 rows)
Sample IDs: ['2_1_2013-04-01', '2_2_2013-04-01', '2_3_2013-04-01', '2_4_2013-04-01', '2_5_2013-04-01']


In [16]:
# Kiểm tra nhanh: Đếm số lượng dòng dự đoán khác 0
non_zero_preds = submission[submission['units'] > 0]
print(f"Số lượng dòng có dự đoán bán hàng: {len(non_zero_preds)}")
print("Ví dụ 5 dòng có số liệu:")
print(non_zero_preds.head())

Số lượng dòng có dự đoán bán hàng: 21762
Ví dụ 5 dòng có số liệu:
                 id       units
4    2_5_2013-04-01   40.250135
10  2_11_2013-04-01    0.431184
43  2_44_2013-04-01  112.111058
84  2_85_2013-04-01    0.031153
92  2_93_2013-04-01    0.151883
